In [45]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time
import csv 

In [46]:
start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"


# Target log path
logs_path = '/data'
writer = tf.summary.FileWriter(logs_path)

# Text file containing words for training
training_file = './data/AAPL.csv'

In [79]:
import pandas as pd
df=pd.read_csv(training_file, sep=',',header=None)
df.shape
#convert pd to numpy array
df=df.values
numpy.delete(df, numpy.s_[1:3], axis=1)   
#p.delete(df, [2], axis=1)  # delete first column
df= numpy.delete(df, (0), axis=0 ) #delete first row
#y = df.astype(np.float32)
df

array([['16:00', '155.30', '17,407,558', '154.97', '155.49', '154.56'],
       ['2017/10/05', '155.3900', '21215870.0000', '154.1800', '155.4400',
        '154.0500'],
       ['2017/10/04', '153.4800', '20088940.0000', '153.6300', '153.8600',
        '152.4600'],
       ..., 
       ['2015/10/07', '110.7800', '46717770.0000', '111.7400', '111.7700',
        '109.4100'],
       ['2015/10/06', '111.3100', '48702200.0000', '110.6300', '111.7400',
        '109.7650'],
       ['2015/10/05', '110.7800', '52033100.0000', '109.8800', '111.3698',
        '109.0700']], dtype=object)

In [83]:
import pandas_datareader as pdr
from datetime import datetime
aapl = pdr.DataReader('AAPL', 'google', '1980-01-01')
aapl

,Open,High,Low,Close,Volume
Date,,,,,
2016-10-10,115.02,116.75,114.72,116.05,36235956
2016-10-11,117.70,118.69,116.20,116.30,64041043
2016-10-12,117.35,117.98,116.75,117.34,37586787
2016-10-13,116.79,117.44,115.72,116.98,35192406
2016-10-14,117.88,118.17,117.13,117.63,35652191
2016-10-17,117.33,117.84,116.78,117.55,23624896
2016-10-18,118.18,118.21,117.45,117.47,24553478
2016-10-19,117.25,117.76,113.80,117.12,20034594
2016-10-20,116.86,117.38,116.33,117.06,24125801


In [87]:
from pylab import *
import pandas as pd
import numpy as np



def Datapull(Stock):
    try:
        df = (pd.io.data.DataReader(Stock,'yahoo',start='01/01/2016'))
        return df
        print ('Retrieved', Stock)
        time.sleep(5)
    except Exception:
        print ('Main Loop', str(e))


def RSIfun(price, n=14):
    delta = price['Close'].diff()
    #-----------
    dUp=
    dDown=

    RolUp=pd.rolling_mean(dUp, n)
    RolDown=pd.rolling_mean(dDown, n).abs()

    RS = RolUp / RolDown
    rsi= 100.0 - (100.0 / (1.0 + RS))
    return rsi

Stock='AAPL'
df=Datapull(Stock)
RSIfun(df)

SyntaxError: invalid syntax (<ipython-input-87-7095ccbc0277>, line 20)

In [ ]:
stock = StockDataFrame.retype(pd.read_csv('stock.csv'))
